In [3]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as st

In [4]:
###### the original dataset without datacleaning

PATH = '../data/rawdata/'
FILES = ['Processed_all species in family level.xlsx', 'Moss-classification.xlsx']

In [5]:
abundance_df = pd.read_excel(PATH+FILES[0])

In [6]:
# Generate new column names
new_column_names = [f"family{i//2 + 1}" if i % 2 == 0 else f"abundance{i//2 + 1}" for i in range(34)]

# Rename the columns
abundance_df.columns = new_column_names

In [62]:
abundance_df.head()

,family1,abundance1,family2,abundance2,family3,abundance3,family4,abundance4,family5,abundance5,...,family13,abundance13,family14,abundance14,family15,abundance15,family16,abundance16,family17,abundance17
0,Beijerinckiaceae,45.833333,Bryum argenteum var. argenteum,8.810919,Kosmotogaceae,36.363636,Xanthobacteraceae,1.198424,Gemmatimonadaceae,4.289914,...,Bryum argenteum var. argenteum,5.457327,Spirochaetaceae,10.051650,Ktedonobacteraceae,8.366511,Mycobacteriaceae,22.903226,Beijerinckiaceae,23.868313
1,Mycobacteriaceae,25.000000,Chloroplast (Order),5.933329,Dysgonomonadaceae,27.272727,Devosiaceae,1.186403,Pseudomonadaceae,2.729285,...,Bryum argenteum var. argenteum,3.978239,Xanthomonadaceae,7.764288,Solibacteraceae (Subgroup 3),2.724834,Beijerinckiaceae,18.709677,Mycobacteriaceae,14.814815
2,Obscuribacterales (Order),20.833333,Chloroplast (Order),1.775470,Spirochaetaceae,18.181818,Micropepsaceae,0.736994,Devosiaceae,1.665880,...,Spirochaetaceae,3.095731,Mycobacteriaceae,3.994500,Ambiguous_taxa,2.710716,Spirochaetaceae,13.870968,Burkholderiaceae,11.934156
3,Ktedonobacteraceae,8.333333,Opitutaceae,1.754333,Prolixibacteraceae,18.181818,Rhodanobacteraceae,0.678737,Burkholderiaceae,1.361013,...,Chloroplast (Order),2.870081,Burkholderiaceae,3.863697,Gimesiaceae,1.993506,Pseudomonadaceae,8.387097,Burkholderiaceae,6.995885
4,Chitinophagaceae,0.000000,Nostocaceae,1.672806,Chitinophagaceae,0.000000,Micropepsaceae,0.650071,Opitutaceae,1.183174,...,Xanthomonadaceae,1.859293,Beijerinckiaceae,3.602093,Acidobacteriales (Order),1.629253,Obscuribacterales (Order),6.129032,Spirochaetaceae,6.584362


In [7]:
abundance_df.shape

(6719, 34)

In [68]:
# Combine dataframes into a list, each list is a sample
abundance_dflist = []
for i in range(1, 18):  # Iterating through each pair
    temp_df = pd.DataFrame({"abundance": abundance_df[f"abundance{i}"].values}, index=abundance_df[f"family{i}"])
    # Drop rows with missing or zero values
    temp_df_cleaned = temp_df[(temp_df[f"abundance"] != 0) & (~temp_df[f"abundance"].isna())]
    # Combine rows by summing values for the same index
    temp_df_final = temp_df_cleaned.groupby(temp_df_cleaned.index).sum()
    temp_df_final = temp_df_final.sort_values(by='abundance', ascending=False)  
    abundance_dflist.append(temp_df_final)


In [70]:
# delete non-family abundance and calculate the remaining to relative abundance
relative_abundance_dflist = []
for i in range(17):
    df = abundance_dflist[i]
    df_family = df[~df.index.str.contains('Order|Phylum|Class')].copy()
    df_family['abundance'] =  df_family['abundance'] /df_family['abundance'].sum()*100
    relative_abundance_dflist.append(df_family)
    

In [72]:
print(abundance_dflist[0].iloc[:20,])
print(relative_abundance_dflist[0].iloc[:20,])
for i in range(17):
    print(relative_abundance_dflist[i].shape[0])

                           abundance
family1                             
Beijerinckiaceae           45.833333
Mycobacteriaceae           25.000000
Obscuribacterales (Order)  20.833333
Ktedonobacteraceae          8.333333
                    abundance
family1                      
Beijerinckiaceae    57.894737
Mycobacteriaceae    31.578947
Ktedonobacteraceae  10.526316
3
149
4
172
140
65
133
135
143
10
173
141
221
188
84
17
23


In [77]:
# combine the abundance from each sample to a dataframe 
relative_abundance_dft = pd.concat(relative_abundance_dflist, axis=1).fillna(0)

relative_abundance_df = relative_abundance_dft.T
relative_abundance_df.index = [f'Sp{i+1}' for i in range(len(relative_abundance_df))]


In [78]:
#
print(relative_abundance_df)

      Beijerinckiaceae  Mycobacteriaceae  Ktedonobacteraceae  \
Sp1          57.894737         31.578947           10.526316   
Sp2           1.448965          0.631692            0.039258   
Sp3           0.000000          0.000000            0.000000   
Sp4           1.247068          0.856460            0.017705   
Sp5           0.524455          1.700505            0.190711   
Sp6           5.468805         80.751917            0.000000   
Sp7           2.525957          1.713743            0.005582   
Sp8           2.184587          1.202563            1.198402   
Sp9           1.280438          0.458262            0.705364   
Sp10          7.142857          7.142857            4.545455   
Sp11          0.997030          0.735948            0.068108   
Sp12          0.683286          1.895930            0.198979   
Sp13          4.018762          2.288878            0.075826   
Sp14          4.194349          4.617800            0.040155   
Sp15          1.897887          2.090615

In [80]:
relative_abundance_df.to_csv('../data/relative_abundance_df.csv', index=True)  # index=False to exclude index from the file